In [5]:
%pwd

'/Users/harvey/Documents/Studies/Computer Science/Deep Learning/HW1P2'

In [3]:
import numpy as np

In [6]:
train = np.load("hw1p2-toy-problem/toy_train_data.npy", allow_pickle=True)

In [9]:
train.shape

(1000,)

In [10]:
type(train)

numpy.ndarray